In [1]:
import json
import pandas as pd
import csv
from pandas import read_excel
import ftputil
import time
from Bio import Entrez
import sys

In [2]:
def get_assembly_id(acc_id):
    """Get Assembly ID from Accession ID"""
    from Bio import Entrez
    handle = Entrez.esearch(db='assembly', term=acc_id)
    record = Entrez.read(handle)
    handle.close()
    assembly = record["IdList"][0]
    return assembly

def get_assembly_summary(acc_id):
    """Get esummary from an assembly ID"""
    from Bio import Entrez
    #provide your own mail here
    Entrez.email =  #email
    assembly = get_assembly_id(acc_id)
    esummary_handle = Entrez.esummary(db="assembly", id=assembly, report="full")
    esummary_record = Entrez.read(esummary_handle)
    genbank_id = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['Synonym']['Genbank']
    refseq_id = esummary_record['DocumentSummarySet']['DocumentSummary'][0]['Synonym']['RefSeq']
    return([assembly, genbank_id, refseq_id])

In [3]:
INPUT_TSV_FILENAME = "../../Bacterial Cultures Catalogue.xlsx"
my_sheet = 'Eggerthella lenta strains'
tsv_df = read_excel(INPUT_TSV_FILENAME, sheet_name = my_sheet)
tsv_df.head()
tsv_df = tsv_df.head()

In [4]:
INPUT_JSON_FILENAME = "../../e_lenta_submission_template.json"

INSTRUMENT_METHOD_LABEL = "Seed Grant TOF"
EXTRACTION_METHOD = "SeedGrant_Extraction"
SAMPLE_PREP = "metagenomic_mouse_fecal"

template_json = json.loads(open(INPUT_JSON_FILENAME).read())

In [5]:
list_acc = tsv_df['Biosample Accession'].tolist()
genbank_dic = {}

for idx,acc_id in enumerate(list_acc):
    try:
        genbank_dic[acc_id] = (get_assembly_summary(acc_id))
    except:
        print("Accesion ID not found: " + acc_id)
        genbank_dic[acc_id] = (['-', '-', '-'])

    print(acc_id)
    print( str(idx+1) + "/" + str(len(list_acc)))

    time.sleep(4)  # Delays for 4 seconds

SAMN08365960
1/5
SAMN08365961
2/5
SAMN08365962
3/5
SAMN08365964
4/5
SAMN08365965
5/5


In [7]:
#print(genbank_dic)
entrez_df = pd.DataFrame(genbank_dic.items(), columns=['Biosample Accession','esummary_record'])
entrez_df[['Assembly_ID', 'GenBank_ID', 'RefSeq_ID']]= pd.DataFrame(entrez_df.esummary_record.values.tolist(), index= entrez_df.index)
entrez_df = entrez_df[['Biosample Accession','Assembly_ID', 'GenBank_ID', 'RefSeq_ID']]
pairing_df = tsv_df.merge(entrez_df, on=['Biosample Accession'])
#print(pairing_df)

{'SAMN08365960': ['1834351', 'GCA_003340245.1', 'GCF_003340245.1'], 'SAMN08365961': ['1834361', 'GCA_003340255.1', 'GCF_003340255.1'], 'SAMN08365962': ['1834341', 'GCA_003340225.1', 'GCF_003340225.1'], 'SAMN08365964': ['1834321', 'GCA_003340195.1', 'GCF_003340195.1'], 'SAMN08365965': ['1834311', 'GCA_003340165.1', 'GCF_003340165.1']}
   Unnamed: 0  Sales order  Item number                   Name   Product  \
0         NaN          NaN          NaN  Eggerthella_lenta_11C  Bacteria   
1         NaN          NaN          NaN  Eggerthella_lenta_14A  Bacteria   
2         NaN          NaN          NaN  Eggerthella_lenta_16A  Bacteria   
3         NaN          NaN          NaN  Eggerthella_lenta_22C  Bacteria   
4         NaN          NaN          NaN  Eggerthella_lenta_28B  Bacteria   

   Level  BEI_accn               Organism Strain_guess  taxon_oid  ...  \
0    NaN       NaN  Eggerthella lenta 11C          11C      84112  ...   
1    NaN       NaN  Eggerthella lenta 14A          14A     

In [13]:
# Preparing the genome tables

all_genome_links = []
for pair in pairing_df.to_dict(orient="records"):
    genome_dict = {}
    genome_dict["genome_label"] = pair["Biosample Accession"]
    genome_dict["genome_ID"] = {
        "genome_type" : "metagenome",
        "GenBank_NCBI_accession" : pair["GenBank_ID"]
    }
    genome_dict["BioSample_accession"] = pair["Biosample Accession"]
    all_genome_links.append(genome_dict)

In [14]:
print(all_genome_links)

[{'genome_label': 'SAMN08365960', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_003340245.1'}, 'BioSample_accession': 'SAMN08365960'}, {'genome_label': 'SAMN08365961', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_003340255.1'}, 'BioSample_accession': 'SAMN08365961'}, {'genome_label': 'SAMN08365962', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_003340225.1'}, 'BioSample_accession': 'SAMN08365962'}, {'genome_label': 'SAMN08365964', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_003340195.1'}, 'BioSample_accession': 'SAMN08365964'}, {'genome_label': 'SAMN08365965', 'genome_ID': {'genome_type': 'metagenome', 'GenBank_NCBI_accession': 'GCA_003340165.1'}, 'BioSample_accession': 'SAMN08365965'}]


In [15]:
# Preparing the join table

all_paired_links = []
for pair in pairing_df.to_dict(orient="records"):
    link_dict = {}
    link_dict["genome_label"] = pair["Biosample Accession"]
    link_dict["metabolomics_file"] = pair["MS data accession, BHI media strains (data in MSV000082045)"]
    link_dict["sample_preparation_label"] = SAMPLE_PREP
    link_dict["extraction_method_label"] = EXTRACTION_METHOD
    link_dict["instrumentation_method_label"] = INSTRUMENT_METHOD_LABEL
    
    all_paired_links.append(link_dict)

In [16]:
# Merging it all together

template_json["genomes"] = all_genome_links
template_json["genome_metabolome_links"] = all_paired_links
with open("../../e_lenta_strains_json.json", "w") as output_file:
    output_file.write(json.dumps(template_json))

In [17]:
print(template_json)

{'version': '1', 'personal': {'submitter_name': 'Alexander Aksenov', 'submitter_orcid': 'https://orcid.org/0000-0002-9445-2248', 'PI_name': 'aaaksenov@ucsd.edu', 'PI_institution': 'University of California, San Diego', 'PI_email': 'pdorrestein@health.ucsd.edu'}, 'metabolomics': {'project': {'GNPSMassIVE_ID': 'MSV000083734', 'MaSSIVE_URL': 'https://gnps.ucsd.edu/ProteoSAFe/result.jsp?task=d80e741723a143348e0f196065427e57&view=advanced_view', 'molecular_network': '85c9922a8b8548e3a537dda24301673f'}}, 'experimental': {'sample_preparation': [{'medium_details': {'medium_type': 'liquid', 'medium': 'other', 'Other_medium': 'Brain Heart Infusion (BHI) Broth', 'Other_medium_link': 'https://microbeonline.com/brain-heart-infusion-bhi-broth-composition-preparation-and-uses/'}, 'growth_parameters': {}, 'aeration': {}, 'sample_preparation_method': 'E Lenta'}], 'extraction_methods': [{'extracted_material': 'cells_supernatant', 'resins': 'Oasis HLB', 'extraction_method': 'Oasis HLB', 'solvents': [{'ra